In [1]:
import os

# Pacotes

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from pathlib import Path

In [3]:
# Tamanho imagem
img_rows, img_cols = 224, 224

input_shape = (img_rows, img_cols, 3)

# Batch size

batch_size = 32

# seed
seed = 42

path_dataset = Path('/scratch/dpfurtado/backup_home/corais/5_subimage_224_224/dataset/1_train_50/')
path_train = path_dataset/'train'
path_train_dev = path_dataset/'train-dev'

#Arquivos npy
path_dir_dataset = './dataset'
npz = path_dir_dataset + '/extracao_efficientnetb0' 

In [4]:
train_dataset = image_dataset_from_directory(
        path_train,
        image_size=(img_rows, img_cols),
        color_mode="rgb",
        batch_size=batch_size,
        shuffle=False) # Manter Shuffle = False, pois não haverá treinamento
train_dev_dataset = image_dataset_from_directory(
        path_train_dev,
        image_size=(img_rows, img_cols),
        color_mode="rgb",
        batch_size=batch_size,
        shuffle=False)

Found 1394 files belonging to 8 classes.
Found 143 files belonging to 8 classes.


2022-05-15 16:04:50.920287: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-15 16:04:50.922350: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-15 16:04:51.837915: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 312.92GiB/s
2022-05-15 16:04:51.838488: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:af:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 312.92GiB/s
2022-05-15 16:04:51.838515: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcu

In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE) #Otimização
train_dev_dataset = train_dev_dataset.prefetch(buffer_size=AUTOTUNE)

In [6]:
cnn = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
inputs = keras.Input(shape=input_shape)
x = preprocess_input(inputs)
x = cnn(x)
output = GlobalAveragePooling2D()(x)
model = Model(inputs, output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 7, 7, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
Total params: 4,049,571
Trainable params: 4,007,548
Non-trainable params: 42,023
_________________________________________________________________


In [7]:
model.save('./model/extrator_cnn/')

2022-05-15 16:05:06.906890: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./model/extrator_cnn/assets


In [8]:
X_train = model.predict(train_dataset)
X_train_dev = model.predict(train_dev_dataset)

2022-05-15 16:05:25.220421: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-15 16:05:25.239376: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2200000000 Hz
2022-05-15 16:05:29.105127: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-05-15 16:05:29.967843: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


In [9]:
print(X_train.shape)
print(X_train_dev.shape)

(1394, 1280)
(143, 1280)


In [10]:
y_train = np.concatenate([y for x, y in train_dataset], axis=0)
y_train_dev = np.concatenate([y for x, y in train_dev_dataset], axis=0)

In [11]:
print(y_train.shape)
print(y_train_dev.shape)

(1394,)
(143,)


In [12]:
np.savez_compressed(npz, X_train=X_train, y_train=y_train, X_train_dev=X_train_dev, y_train_dev=y_train_dev)